# This notebook is made to test out Claude from Anthropic.
### The code below leverages AWS Bedrock services to access Claude.



*   Make sure you use ENV variables instead of hardcoding
*   Do not commit/push this notebook with env variables
*   Have Fun!




#### Installing dependencies

In [ ]:
!pip install boto3

#### Importing packages and environment variables

In [34]:
import os
import boto3
import json
import base64

# Set an environment variable here
os.environ['AWS_ACCESS_KEY_ID'] = 'Aws access key id'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'Aws secret access key'
os.environ['REGION_NAME'] = 'Region from aws account, find it through IAM'
os.environ['AWS_RUNTIME'] = 'Service runtime'

# Saving env values in variables
AWS_ACCESS_KEY_ID = os.getenv('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = os.getenv('AWS_SECRET_ACCESS_KEY')
REGION_NAME = os.getenv('REGION_NAME')
AWS_RUNTIME = os.getenv('AWS_RUNTIME')

#### Initializing boto3 client using runtime and aws keys


*   `AWS_RUNTIME`: Determines the service we want to use.
*   `AWS_ACCESS_KEY_ID`: Account access key Id from AWS. (AWS console > IAM > Dashboard > "Manage Access Keys")
*   `AWS_SECRET_ACCESS_KEY`: Account seceret access key. (Get from IAM)
*   `REGION_NAME`: Default configured region name from AWS account

In [35]:
client = boto3.client(
    AWS_RUNTIME,
    aws_access_key_id = AWS_ACCESS_KEY_ID,
    aws_secret_access_key = AWS_SECRET_ACCESS_KEY,
    region_name= REGION_NAME
)

#### Creating variable `input_data` to pass into the `invoke_client()` method from boto3 client initialized in the previous step.

Inside `input_data` we can specify all the model parameters and prompt:


*   `modelId`: Defunes which model to use from bedrock
*   `contentType`: Defines the response type, default type is `JSON`.
*   `accept`: Defines the input type, default type is `String`.
*   `body`: Body objects defines multiple parameters:

    1.   **prompt**: Takes in the user prompt.
    2.   **max_tokens_to_sample**: Defines the maximum output tokens.
    3.   **temperature**: Defines the temperature for the model output.
    4.   **top_k**: Sample from the k most likely next tokens at each step. Lower k focuses on higher probability tokens.
    5.   **top_p**: The cumulative probability cutoff for token selection. Lower values mean sampling from a smaller, more top-weighted nucleus.
    6.   **anthropic_version**: Version of the deployed model on bedrock.






In [36]:
# Specify the input data and model ID
input_data = {
  "modelId": "anthropic.claude-v2",
  "contentType": "application/json",
  "accept": "*/*",
  "body": {
    "prompt": "\n\nHuman: write poem to make me happy!\n\nAssistant:",
    "max_tokens_to_sample": 300,
    "temperature": 0.5,
    "top_k": 250,
    "top_p": 1,
    "anthropic_version": "bedrock-2023-05-31"
  }
}

#### Using the boto3 client initialized earlier to invoke the deployed models using the `invoke_model()` methon.

* The below code uses `invoke_model_with_response_stream()` method to recive data in chunks, which means we can yield these chunks in order to stream responses from the llm.

* You can also use `invoke_model()` and pass in the same parameters to get the full response.


In [ ]:
# Invoke the model for inference
response = client.invoke_model_with_response_stream(contentType=input_data["contentType"], body=json.dumps(input_data['body']), modelId=input_data['modelId'])
event_stream = response['body']

# Iterate through events in the stream
for event in event_stream:
    # Event messages might be bytes, requiring decoding, and potentially deserialization from JSON
    print(event['chunk'])